<a href="https://colab.research.google.com/github/dlsyscourse/lecture14/blob/main/14_hardware_acceleration_architecture_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 14: Hardware Acceleration Implementation

In this lecture, we will to walk through backend scafoldings to get us hardware accelerations for needle.




## Select a GPU runtime type
In this lecture, we are going to make use of c++ and CUDA to build accelerated linear algebra libraries. In order to do so, please make sure you select a runtime type with GPU and rerun the cells if needed:
- Click on the "Runtime" tab
- Click "Change runtime type"
- Select GPU

After you started the right runtime, you can run the following command to check if there is a GPU available.

In [27]:
!nvidia-smi

Mon Oct  9 14:55:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    33W /  70W |    105MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Prepare the codebase

To get started, we can clone the related repo from the github.

In [3]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!mkdir -p 10714f23
%cd /content/drive/MyDrive/10714f23
# comment out the following line if you run it for the second time
# as you already have a local copy of lecture14
# !git clone https://github.com/dlsyscourse/lecture14
!rm -rf /content/needle
!ln -s /content/drive/MyDrive/10714f23/lecture14 /content/needle

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive
/content/drive/MyDrive/10714f23
Cloning into 'lecture14'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 96 (delta 29), reused 87 (delta 24), pack-reused 0
Receiving objects: 100% (96/96), 49.00 KiB | 1.69 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [1]:
!python3 -m pip install pybind11

### Build the needle cuda library

We leverage pybind to build a c++/cuda library for acceleration. You can type make to build the corresponding library.

In [4]:
%cd /content/needle
!make clean
!make

/content/drive/MyDrive/10714f23/lecture14
rm -rf build python/needle/backend_ndarray/ndarray_backend*.so
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Python: /usr/local/bin/python (found version "3.10.12") found components: Development Interpreter Development.Module Development.Embed 
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- Found pybind11: /usr/local/lib/python3.10/dist-packages/pybind11/include (found version "2.11.1")
CMake Warning (dev) at CMakeLists.txt:55 (find_package):
  Policy CMP0146 is n

We can then run the following command to make the path to the package available in colab's environment as well as the PYTHONPATH.

In [5]:
%set_env PYTHONPATH /content/needle/python:/env/python
import sys
sys.path.append("/content/needle/python")

env: PYTHONPATH=/content/needle/python:/env/python


## Codebase walkthrough


Now click the files panel on the left side. You should be able to see these files

Python:
- needle/backend_ndarray/ndarray.py
- needle/backend_ndarray/ndarray_backend_numpy.py

C++/CUDA
- src/ndarray_backend_cpu.cc
- src/ndarray_backend_cuda.cu

The main goal of this lecture is to create an accelerated ndarray library.
As a result, we do not need to deal with needle.Tensor for now and will focus on backend_ndarray's implementation.

After we build up this array library, we can then use it to power backend array computations in needle.


## Creating a CUDA NDArray






In [6]:
from needle import backend_ndarray as nd

Using needle backend


In [7]:
x = nd.NDArray([1, 2, 3], device=nd.cpu())

In [8]:
x = nd.NDArray([1, 2, 3], device=nd.cuda())

In [9]:
x.device

cuda()

In [10]:
x = nd.NDArray([1,2,3], device=nd.cuda())

In [11]:
y = x + 1

In [12]:
y

NDArray([2. 3. 4.], device=cuda())

In [13]:
y = x + x

In [14]:
y

NDArray([2. 4. 6.], device=cuda())

We can create a CUDA tensor from the data by specifying a device keyword.

In [15]:
x = nd.NDArray([1, 2, 3], device=nd.cuda())

In [16]:
y = x + 1

In [17]:
x.numpy()

array([1., 2., 3.], dtype=float32)

In [18]:
x.device

cuda()

In [19]:
y = x + 1

In [20]:
y.device

cuda()

In [21]:
y.numpy()

array([2., 3., 4.], dtype=float32)

### Key Data Structures

Key data structures in backend_ndarray

- NDArray: the container to hold device specific ndarray
- BackendDevice: backend device
    - mod holds the module implementation that implements all functions
    - checkout ndarray_backend_numpy.py for a python-side reference.



## Trace GPU execution

Now, let us take a look at what happens when we execute the following code


In [22]:
x = nd.NDArray([1, 2, 3], device=nd.cuda())
y = x + 1

In [24]:
x.device.from_numpy

<function needle.backend_ndarray.ndarray_backend_numpy.from_numpy(a, out)>

In [23]:
x = nd.NDArray([1, 2, 3])

In [25]:
x.device.from_numpy

<function needle.backend_ndarray.ndarray_backend_numpy.from_numpy(a, out)>

Have the following trace:

backend_ndarray/ndarray.py
- `NDArray.__add__`
- `NDArray.ewise_or_scalar`
- `ndarray_backend_cpu.cc:ScalarAdd`

In [26]:
y.numpy()

array([2., 3., 4.], dtype=float32)

Have the following trace:

- `NDArray.numpy`
- `ndarray_backend_cpu.cc:to_numpy`

## Guidelines for Reading C++/CUDA related Files

Read
- src/ndarray_backend_cpu.cc
- src/ndarray_backend_cuda.cu


Optional
- CMakeLists.txt: this is used to setup the build and likely you do not need to tweak it.







## NDArray Data Structure

Open up `python/needle/backend_ndarray/ndarray.py`.

An NDArray contains the following fields:
- handle: The backend handle that build a flat array which stores the data.
- shape: The shape of the NDArray
- strides: The strides that shows how do we access multi-dimensional elements
- offset: The offset of the first element.
- device: The backend device that backs the computation






## Transformation as Strided Computation

We can leverage the strides and offset to perform transform/slicing with zero copy.

- Broadcast: insert strides that equals 0
- Tranpose: swap the strides
- Slice: change the offset and shape

For most of the computations, however, we will call `array.compact()` first to get a contiguous and aligned memory before running the computation.

In [27]:
import numpy as np
x = nd.NDArray([0,1,2,3,4,5], device=nd.cpu_numpy())

In [28]:
x.numpy()

array([0., 1., 2., 3., 4., 5.], dtype=float32)

In [29]:
z = nd.NDArray.make(shape=(2, 3),
                strides=(3, 1),
                device=x.device,
                handle=x._handle,
                offset=0)
z

NDArray([[0. 1. 2.]
 [3. 4. 5.]], device=cpu_numpy())

In [30]:
b = nd.NDArray.make(shape=(2, 3, 4),
                    strides=(3, 1, 0),
                    device=z.device,
                    handle=z._handle,
                    offset=0)
b

NDArray([[[0. 0. 0. 0.]
  [1. 1. 1. 1.]
  [2. 2. 2. 2.]]

 [[3. 3. 3. 3.]
  [4. 4. 4. 4.]
  [5. 5. 5. 5.]]], device=cpu_numpy())

In [31]:
y = nd.NDArray.make(shape=(3, 2, 2), strides=(2, 1, 0), device=x.device, handle=x._handle, offset=0)
y.numpy()

array([[[0., 0.],
        [1., 1.]],

       [[2., 2.],
        [3., 3.]],

       [[4., 4.],
        [5., 5.]]], dtype=float32)

In [32]:
x = nd.NDArray([1, 2, 3, 4], device=nd.cpu_numpy())

In [33]:
x.numpy()

array([1., 2., 3., 4.], dtype=float32)

We can use strides and shape manipulation to create different views of the same array.

In [34]:
y = nd.NDArray.make(shape=(2, 2), strides=(2, 1), device=x.device, handle=x._handle, offset=0)

In [35]:
y.numpy()

array([[1., 2.],
       [3., 4.]], dtype=float32)

In [36]:
z = nd.NDArray.make(shape=(2, 1), strides=(2, 1), device=x.device, handle=x._handle, offset=1)

In [37]:
z.numpy()

array([[2.],
       [4.]], dtype=float32)

## CUDA Acceleration

Now let us open `src/ndarray_cuda_backend.cu` and take a look at current implementation of GPU ops.


## Steps for adding a new operator implementation
- Add an implementation in `ndarray_backend_cuda.cu`, expose via pybind
- Call into the operator in ndarray.py
- Write up testcases

In [38]:
!make

-- Found pybind11: /usr/local/lib/python3.10/dist-packages/pybind11/include (found version "2.11.1")
CMake Warning (dev) at CMakeLists.txt:55 (find_package):
  Policy CMP0146 is not set: The FindCUDA module is removed.  Run "cmake
  --help-policy CMP0146" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning.

This warning is for project developers.  Use -Wno-dev to suppress it.

-- Found cuda, building cuda backend
Mon Oct  9 15:11:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|=========================

If we directly run the code block, we will see an error, because ewise mul is not yet implemented

In [39]:
x = nd.NDArray([1,2,3], device=nd.cuda())
x * 2

AttributeError: ignored

In [ ]:
!nvprof python test_mul.py

==1619== NVPROF is profiling process 1619, command: python3 test_mul.py
[3. 6. 9.]
==1619== Profiling application: python3 test_mul.py
==1619== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   53.55%  4.8320us         1  4.8320us  4.8320us  4.8320us  needle::cuda::ScalarMulKernel(float const *, float, float*, unsigned long)
                   23.40%  2.1120us         1  2.1120us  2.1120us  2.1120us  [CUDA memcpy DtoH]
                   23.05%  2.0800us         1  2.0800us  2.0800us  2.0800us  [CUDA memcpy HtoD]
      API calls:   99.74%  333.65ms         2  166.83ms  9.9830us  333.64ms  cudaMalloc
                    0.13%  421.20us         1  421.20us  421.20us  421.20us  cuDeviceTotalMem
                    0.05%  165.97us         2  82.987us  19.197us  146.78us  cudaFree
                    0.05%  158.40us       101  1.5680us     140ns  67.406us  cuDeviceGetAttribute
                    0.02%  53.548us         2 

## Connect back to needle Tensor

So far we only played with the `backend_ndarray` subpackage, which is a self-contained ndarray implementation within needle.

We can connect the ndarray back to needle as a backend.

In [40]:
import needle as ndl

In [41]:
x = ndl.Tensor([1,2,3], device=nd.cuda(), dtype="float32")
y = ndl.Tensor([2,3,5], device=nd.cuda(), dtype="float32")
z = x + y
z

needle.Tensor([3. 5. 8.])

In [42]:
z.device

cuda()

In [43]:
type(z.cached_data)

needle.backend_ndarray.ndarray.NDArray

## Write Standalone Python Test Files

Now that we have additional c++/cuda libraries in needle, we will need to type make in order to rebuild the library. Additionally, because the colab environment caches the old library, it is inconvenient to use the ipython cells to debug the updated library.




In [44]:
!make

-- Found pybind11: /usr/local/lib/python3.10/dist-packages/pybind11/include (found version "2.11.1")
CMake Warning (dev) at CMakeLists.txt:55 (find_package):
  Policy CMP0146 is not set: The FindCUDA module is removed.  Run "cmake
  --help-policy CMP0146" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning.

This warning is for project developers.  Use -Wno-dev to suppress it.

-- Found cuda, building cuda backend
Mon Oct  9 15:12:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|=========================


We recommend writing separate python files and invoke them from the command line. Create a new file `tests/mytest.py` and write your local tests. This is also a common develop practice in big projects that involves python c++ FFI.

In [45]:
!python tests/mytest.py

python3: can't open file '/content/drive/MyDrive/10714f23/lecture14/tests/mytest.py': [Errno 2] No such file or directory


After we have building the library, we could choose to fully restart the runtime (factory reset runtime) if you want to bring the updated change back to another colab. Note that you will need to save your code changes to the drive or a private github repo.